In [2]:
# --- Setup & Imports ---
import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from typing import Dict, Any, List, Tuple, Callable, Optional
from dataclasses import dataclass
import json

# --- 1. Pfad-Setup ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# --- Import necessary functions and models ---
from src.config import GlobalConfig, DEFAULT_CORR_SPEC, OUTPUTS # Import OUTPUTS
from src.io_timesplits import load_target, load_ifo_features, _clean_column_name # Import cleaner
from src.features import (
    select_lags_per_feature, build_engineered_matrix,
    screen_k1, redundancy_reduce_greedy,
    _nuisance_matrix, _residualize_vec, pw_corr, _align_D_to_taus
)
from src.models.EN import ForecastModel as ENModel
from src.models.ET import ForecastModel as ETModel
from src.models.LGBM import ForecastModel as LGBMModel

print('PROJECT_ROOT =', PROJECT_ROOT)
OUTPUT_DIR = OUTPUTS / "feature_importance" # Specific output folder
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print('Output Directory =', OUTPUT_DIR)


# --- 2. Konfiguration ---
REDUNDANCY_THRESHOLD = 0.90  # Schwelle für den finalen Redundanzfilter (für Mean Importance)
FINAL_TOP_K_MEAN = 20      # Gewünschte Anzahl diverser Features (für Mean Importance)
TOP_N_PER_MODEL = 20       # Top N für die Schnittmengen-Methode

# --- Feste Hyperparameter für die Importance-Modelle ---
HP_EN = {'alpha': 0.5}
HP_ET = {'n_estimators': 100, 'max_features': 0.1, 'min_samples_leaf': 10}
HP_LGBM = {'n_estimators': 100, 'learning_rate': 0.1, 'num_leaves': 31,
           'colsample_bytree': 0.1, 'min_child_samples': 20,
           'early_stopping_rounds': 10, 'es_val_tail_size': 12}

# --- 3. Daten laden ---
y = load_target()
# Load IFO features using the function that already cleans names
X_ifo = load_ifo_features()

idx = y.index.intersection(X_ifo.index)
y, X_ifo = y.loc[idx], X_ifo.loc[idx]
T = len(y)
print('Daten geladen. Shapes:', X_ifo.shape, y.shape)

# --- 4. Modifizierte Feature-Pipeline (nur bester Lag, kein RM3, kein Top-K) ---
def run_feature_pipeline_for_importance(
    X: pd.DataFrame, y: pd.Series, t_origin: int
) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Führt FE durch (nur bester Lag), aber KEIN RM3, KEIN Top-K Screening, KEINEN Redundanzfilter/DR.
    Gibt die resultierende Feature-Matrix und den Zielvektor für den Trainingszeitraum zurück.
    Stellt sicher, dass Spaltennamen bereinigt sind.
    """
    I_t = t_origin + 1
    corr_spec = DEFAULT_CORR_SPEC

    lag_candidates = tuple(range(0, 13))
    top_k_lags = 1
    use_rm3 = False

    # 1) Lag-Selektion
    lag_map, _, D, taus = select_lags_per_feature(
        X, y, I_t=I_t, L=lag_candidates, k=top_k_lags, corr_spec=corr_spec
    )

    # 2) Feature Engineering
    X_eng = build_engineered_matrix(X, lag_map)
    if X_eng.empty:
         return pd.DataFrame(), pd.Series(dtype=float)

    # 3) Head-Trim
    def _lag_of(col):
        try: return int(str(col).split('__lag')[-1])
        except: return 0
    head_needed = max([_lag_of(c) for c in X_eng.columns] + [0])
    taus_model = taus[taus - head_needed >= 0] if head_needed > 0 else taus
    if len(taus_model) == 0:
        taus_model = taus[-1:].copy()
        if head_needed > taus_model[0]:
             return pd.DataFrame(), pd.Series(dtype=float)

    X_train_final = X_eng.iloc[taus_model, :].copy()
    y_train_final = y.shift(-1).iloc[taus_model].copy()
    if X_train_final.empty:
        return pd.DataFrame(), pd.Series(dtype=float)

    # NaN/Inf sicherstellen
    X_train_final = X_train_final.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # --- Spaltennamen proaktiv bereinigen (redundant, da load_ifo_features es schon tut, aber sicher) ---
    original_cols = X_train_final.columns.tolist()
    cleaned_cols = [_clean_column_name(col) for col in original_cols]
    if original_cols != cleaned_cols:
        X_train_final.columns = cleaned_cols
    # --- ENDE ---

    return X_train_final, y_train_final

# --- 5. Helper Function für Rolling Importance & Aggregation ---
def perform_rolling_importance(start_origin: int, end_origin: int) -> pd.DataFrame:
    """Führt die Rolling Importance Schleife durch und gibt aggregierte, NORMALISIERTE Wichtigkeit zurück."""
    all_importances = {'EN': [], 'ET': [], 'LGBM': []}
    origins = range(start_origin, end_origin + 1)
    all_feature_names_seen = set() # Track all features encountered

    for t in tqdm(origins, desc=f"Rolling Importance ({start_origin}-{end_origin})"):
        if t >= T - 1: break

        X_tr, y_tr = run_feature_pipeline_for_importance(X_ifo, y, t)

        if X_tr.empty or y_tr.empty:
            print(f"Skipping origin t={t} due to empty data after pipeline.")
            continue

        feature_names = X_tr.columns.tolist()
        all_feature_names_seen.update(feature_names) # Add current features to set

        results_t = {'origin': t, 'date': y.index[t]}

        # Elastic Net
        try:
            model_en = ENModel(HP_EN).fit(X_tr.copy(), y_tr.copy()) # Use copies for safety
            coefs = model_en._reg.coef_
            results_t['EN'] = dict(zip(feature_names, np.abs(coefs)))
        except Exception as e:
            print(f"EN failed at t={t}: {e}")
            results_t['EN'] = {}

        # Extra Trees
        try:
            model_et = ETModel(HP_ET).fit(X_tr.copy(), y_tr.copy())
            results_t['ET'] = model_et.get_feature_importances()
        except Exception as e:
            print(f"ET failed at t={t}: {e}")
            results_t['ET'] = {}

        # LightGBM
        try:
            model_lgbm = LGBMModel(HP_LGBM).fit(X_tr.copy(), y_tr.copy())
            results_t['LGBM'] = model_lgbm.get_feature_importances()
        except Exception as e:
            print(f"LGBM failed at t={t}: {e}")
            results_t['LGBM'] = {}

        all_importances['EN'].append(results_t.get('EN', {}))
        all_importances['ET'].append(results_t.get('ET', {}))
        all_importances['LGBM'].append(results_t.get('LGBM', {}))

    print(f"Rolling Importance Calculation ({start_origin}-{end_origin}) Done.")

    # --- Aggregation über die Zeit (ROBUSTER) ---
    agg_importance = {}
    valid_models = [m for m, imp_list in all_importances.items() if imp_list]

    if not valid_models:
        print("Warning: No importance results generated for any model.")
        return pd.DataFrame()

    # Create DataFrame with all features seen across all time steps
    df_agg = pd.DataFrame(index=sorted(list(all_feature_names_seen)))

    for model_name in valid_models:
        # Create DataFrame ensuring all features are present (fillna=0)
        df_imp = pd.DataFrame(all_importances[model_name]).fillna(0.0)
        # Reindex to include all features seen anywhere, fill missing with 0
        df_imp = df_imp.reindex(columns=df_agg.index, fill_value=0.0)
        # Calculate mean importance for this model
        df_agg[model_name] = df_imp.mean(axis=0)

    # Normalize per model
    df_agg_norm = df_agg.div(df_agg.sum(axis=0), axis=1).fillna(0.0) # Ensure division by zero doesn't cause NaN
    df_agg_norm['Mean_Importance'] = df_agg_norm.mean(axis=1)

    return df_agg_norm

# --- 6. Helper Function für finale Top-K Auswahl (mittels Redundanzfilter) ---
def select_diverse_top_k(df_agg_norm: pd.DataFrame, redundancy_threshold: float, final_top_k: int, t_ref: int) -> List[str]:
    """Wählt die Top K diversen Features aus der aggregierten Importance (Mean)."""
    if df_agg_norm.empty: return []

    # Schritt A: Wähle Top N relevante Features
    N = min(len(df_agg_norm), 200)
    top_n_features = df_agg_norm.sort_values('Mean_Importance', ascending=False).head(N).index.tolist()
    print(f"Top {len(top_n_features)} Kandidaten basierend auf mittlerer Wichtigkeit ausgewählt.")

    # Schritt B: Wende Redundanzfilter an
    X_tr_ref, _ = run_feature_pipeline_for_importance(X_ifo, y, t_ref)

    valid_top_n = [f for f in top_n_features if f in X_tr_ref.columns]
    if not valid_top_n:
        print("Warnung: Keine der Top-N Features im Referenz-DataFrame gefunden.")
        return []
    X_top_n = X_tr_ref[valid_top_n].copy()

    # Nuisance Matrix D basierend auf Indizes von X_top_n
    original_indices = X_top_n.index
    y_lag1_aligned = y.shift(1).reindex(original_indices).fillna(0.0)
    D_top_n = np.vstack([np.ones(len(X_top_n)), y_lag1_aligned.values]).T
    local_taus = np.arange(len(X_top_n))

    # Sortiere nach Wichtigkeit
    scores_top_n = df_agg_norm.loc[valid_top_n, 'Mean_Importance'].to_dict()

    diverse_features = redundancy_reduce_greedy(
        X_sel=X_top_n,
        corr_spec=DEFAULT_CORR_SPEC,
        D=D_top_n,
        taus=local_taus,
        redundancy_param=redundancy_threshold,
        scores=scores_top_n
    )

    final_top_features = diverse_features[:final_top_k]
    return final_top_features

# ======================================================================
# --- RUN 1: Ex-Ante Analyse (t=180 to 239) ---
# ======================================================================
print("\n" + "="*20)
print("--- STARTING EX-ANTE ANALYSIS (t=180 to 239) ---")
print("="*20)
START_ORIGIN_EX_ANTE = 180
END_ORIGIN_EX_ANTE = 239

df_agg_norm_ex_ante = perform_rolling_importance(START_ORIGIN_EX_ANTE, END_ORIGIN_EX_ANTE)

# --- Output 1: Top Features All (Intersection) ---
if not df_agg_norm_ex_ante.empty:
    top_en   = set(df_agg_norm_ex_ante.sort_values('EN', ascending=False).head(TOP_N_PER_MODEL).index)
    top_et   = set(df_agg_norm_ex_ante.sort_values('ET', ascending=False).head(TOP_N_PER_MODEL).index)
    top_lgbm = set(df_agg_norm_ex_ante.sort_values('LGBM', ascending=False).head(TOP_N_PER_MODEL).index)

    intersect_features_ex_ante = sorted(list(top_en.intersection(top_et).intersection(top_lgbm)))

    print(f"\n--- Top Features Ex-Ante (Intersection of Top {TOP_N_PER_MODEL} per model) ---")
    print(f"Anzahl: {len(intersect_features_ex_ante)}")
    for i, feat in enumerate(intersect_features_ex_ante):
        print(f"{i+1}. {feat}")

    # Save Intersection List
    output_path_intersect_ante = OUTPUT_DIR / "top_features_all_ex_ante.json"
    with open(output_path_intersect_ante, "w") as f:
        json.dump(intersect_features_ex_ante, f, indent=2)
    print(f"Saved intersection list to: {output_path_intersect_ante}")

# --- Output 2: Top Mean Features (Diverse) ---
final_top_features_mean_ex_ante = select_diverse_top_k(
    df_agg_norm_ex_ante, REDUNDANCY_THRESHOLD, FINAL_TOP_K_MEAN, t_ref=END_ORIGIN_EX_ANTE
)

print(f"\n--- Finale Top {len(final_top_features_mean_ex_ante)} Ex-Ante Diverse Features (Mean Importance) ---")
for i, feat in enumerate(final_top_features_mean_ex_ante):
    print(f"{i+1}. {feat}")

# Save Mean Diverse List
output_path_mean_ante = OUTPUT_DIR / "top_mean_features_ex_ante.json"
with open(output_path_mean_ante, "w") as f:
    json.dump(final_top_features_mean_ex_ante, f, indent=2)
print(f"Saved mean diverse list to: {output_path_mean_ante}")


# ======================================================================
# --- RUN 2: Ex-Post Analyse (t=240 to T-1) ---
# ======================================================================
print("\n" + "="*50)
print("--- STARTING EX-POST ANALYSIS (t=240 to T-1) ---")
print("="*50)
START_ORIGIN_EX_POST = 240
END_ORIGIN_EX_POST = T - 2

df_agg_norm_ex_post = perform_rolling_importance(START_ORIGIN_EX_POST, END_ORIGIN_EX_POST)

# --- Output 1: Top Features All (Intersection) ---
if not df_agg_norm_ex_post.empty:
    top_en_post   = set(df_agg_norm_ex_post.sort_values('EN', ascending=False).head(TOP_N_PER_MODEL).index)
    top_et_post   = set(df_agg_norm_ex_post.sort_values('ET', ascending=False).head(TOP_N_PER_MODEL).index)
    top_lgbm_post = set(df_agg_norm_ex_post.sort_values('LGBM', ascending=False).head(TOP_N_PER_MODEL).index)

    intersect_features_ex_post = sorted(list(top_en_post.intersection(top_et_post).intersection(top_lgbm_post)))

    print(f"\n--- Top Features Ex-Post (Intersection of Top {TOP_N_PER_MODEL} per model) ---")
    print(f"Anzahl: {len(intersect_features_ex_post)}")
    for i, feat in enumerate(intersect_features_ex_post):
        print(f"{i+1}. {feat}")

    # Save Intersection List
    output_path_intersect_post = OUTPUT_DIR / "top_features_all_ex_post.json"
    with open(output_path_intersect_post, "w") as f:
        json.dump(intersect_features_ex_post, f, indent=2)
    print(f"Saved intersection list to: {output_path_intersect_post}")


# --- Output 2: Top Mean Features (Diverse) ---
final_top_features_mean_ex_post = select_diverse_top_k(
    df_agg_norm_ex_post, REDUNDANCY_THRESHOLD, FINAL_TOP_K_MEAN, t_ref=END_ORIGIN_EX_POST
)

print(f"\n--- Finale Top {len(final_top_features_mean_ex_post)} Ex-Post Diverse Features (Mean Importance) ---")
for i, feat in enumerate(final_top_features_mean_ex_post):
    print(f"{i+1}. {feat}")

# Save Mean Diverse List
output_path_mean_post = OUTPUT_DIR / "top_mean_features_ex_post.json"
with open(output_path_mean_post, "w") as f:
    json.dump(final_top_features_mean_ex_post, f, indent=2)
print(f"Saved mean diverse list to: {output_path_mean_post}")

print("\nFeature Importance Analysis Complete.")

PROJECT_ROOT = /Users/jonasschernich/Documents/Masterarbeit/Code
Output Directory = /Users/jonasschernich/Documents/Masterarbeit/Code/outputs/feature_importance
INFO in load_ifo_features: Renaming columns to ensure validity.
Daten geladen. Shapes: (407, 20) (407,)

--- STARTING EX-ANTE ANALYSIS (t=180 to 239) ---


Rolling Importance (180-239):   2%|▏         | 1/60 [00:00<00:06,  9.78it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000022 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1080
[LightGBM] [Info] Number of data points in the train set: 157, number of used features: 20
[LightGBM] [Info] Start training from score 0.040810
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

Rolling Importance (180-239):   5%|▌         | 3/60 [00:00<00:05, 11.01it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1087
[LightGBM] [Info] Number of data points in the train set: 158, number of used features: 20
[LightGBM] [Info] Start training from score 0.050108
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000031 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2509
[LightGBM] [Info] Number of data points in the train set: 374, number of used features: 20
[LightGBM] [Info] Start training from score 0.068190
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

Rolling Importance (240-405):  98%|█████████▊| 162/166 [00:14<00:00, 13.32it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000032 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2534
[LightGBM] [Info] Number of data points in the train set: 378, number of used features: 20
[LightGBM] [Info] Start training from score 0.066307
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

Rolling Importance (240-405): 100%|██████████| 166/166 [00:14<00:00, 11.45it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000032 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2555
[LightGBM] [Info] Number of data points in the train set: 381, number of used features: 20
[LightGBM] [Info] Start training from score 0.061134
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b